# RECAP Socket + Make it better

### Client

In [ ]:
"""Client socket"""

import socket

HOST = '127.0.0.1' # server IP
PORT = 12345 # server port

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    print("Connected to server")

    while True:
        message = input("Enter a message('exit' to terminated): ")
        s.send((message+"\n").encode())

        if message == "exit":
            break

        message_received = ""
        while True:
            data = s.recv(1024)
            if data:
                print('Received data chunk from server: ', repr(data))
                message_received += data.decode()

                if message_received.endswith("\n"):
                    break
            else:
                print("Connection lost!")
                break

        if message_received:
            print(message_received)
        else:
            break

print("Client finished")


### Server (self-close)

In [ ]:
"""Server socket"""

import socket
import sys
# from pyexpat.errors import messages

HOST = '0.0.0.0' # accept connections from any IP
PORT = 12345 # port to listen on
s = None
close = False # for checking if server should close

# create socket
try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print('Socket created')
except OSError as msg:
    s = None
    print(f'Error creating socket: {msg}')
    sys.exit(1)

# bind and listen
try:
    s.bind((HOST, PORT))
    s.listen()
    print('Socket bound and listening')
except OSError as msg:
    print('Error binding/listening!')
    s.close()
    sys.exit(1)

while True:
    print('Wating for connection...')

    conn, addr = s.accept()

    with conn:
        print('Connection accepted from ', addr)
        while True:
            message_received = ''

            while True:
                data = conn.recv(1024)
                if data:
                    if data.decode() == 'exit\n':
                        break
                    print('Received data chunk from client: ', repr(data))
                    message_received += data.decode()
                    if message_received.endswith('\n'):
                        break
                else:
                    print('Connection lost!')
                    break

            if message_received:
                print('Received message: ', message_received)
                server_msg = input('Enter message to send to client: ')
                if server_msg == 'exit':
                    close = True
                    break
                # conn.send(('Server summarized: ' + message_received[:10] + '\n').encode())
                conn.send(('Message from server: ' + server_msg + '\n').encode())
            else:
                break

        if close:
            break

s.close()
print('Server finished')


### Server (Close when client exit)

In [ ]:
"""Server socket"""

import socket
import sys
# from pyexpat.errors import messages

HOST = '0.0.0.0' # accept connections from any IP
PORT = 12345 # port to listen on
s = None

# create socket
try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print('Socket created')
except OSError as msg:
    s = None
    print(f'Error creating socket: {msg}')
    sys.exit(1)

# bind and listen
try:
    s.bind((HOST, PORT))
    s.listen()
    print('Socket bound and listening')
except OSError as msg:
    print('Error binding/listening!')
    s.close()
    sys.exit(1)

conn, addr = s.accept()

with conn:
    print('Connection accepted from ', addr)
    while True:
        message_received = ''

        while True:
            data = conn.recv(1024)
            if data:
                if data.decode() == 'exit\n':
                    break
                print('Received data chunk from client: ', repr(data))
                message_received += data.decode()
                if message_received.endswith('\n'):
                    break
            else:
                print('Connection lost!')
                break

        if message_received:
            print('Received message: ', message_received)
            server_msg = input('Enter message to send to client: ')
            # conn.send(('Server summarized: ' + message_received[:10] + '\n').encode())
            conn.send(('Message from server: ' + server_msg + '\n').encode())
        else:
            break

s.close()
print('Server finished')


# Threading

- With threading, we can perform parallel programming.
- Run this code in .py file to see the result

In [ ]:
"""Threading example"""

import threading

def print_numbers():
    for i in range(10):
        print(i, end=' ')

def print_letters():
    for i in range(65, 75):
        print(chr(i), end=' ')

t1 = threading.Thread(target=print_numbers)
t2 = threading.Thread(target=print_letters)

t1.start()
t2.start()

t1.join()
t2.join()

print()
print('Thread done!')

## Socket with Threading

In [ ]:
"""Client socket with Threading"""

import socket
import threading

HOST = '127.0.0.1'
PORT = 21002

def send_message_function(client_socket):
    while True:
        message = input("Enter a message: ")
        client_socket.send((message + "\n").encode())

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    print("Connected to server")

    send_thread = threading.Thread(target=send_message_function, args=(s,))
    send_thread.start()

    while True:
        # message = input("Enter a message: ")
        # s.send((message+"\n").encode())

        message_received = ""
        while True:
            data = s.recv(32)
            if data:
                print('Received data chunk from server: ', repr(data))
                message_received += data.decode()
                if message_received.endswith("\n"):
                    print("End of message received")
                    break
            else:
                print("Connection lost!")
                break
        if not message_received:
            break

        print("Received message: ", message_received)

print("Client finished")
